In [9]:
from utils.spark_config import get_spark_session
from configs.paths import ROOT_DIR

spark = get_spark_session()
spark

25/08/21 12:11:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [10]:
churn = spark.sql("SELECT * FROM spark_catalog.delta.`/home/administrator/Desktop/datascience/github/learning-spark/data/gold/customer_churn_gold`")
churn.show()

25/08/21 12:11:09 WARN ObjectStore: Failed to get database delta, returning NoSuchObjectException


+----------+-----+-------------------------------------+-----------------------------+----------------------------+---------------------------------+---------------------------+---------------------------+----------------------------------------+------------------------------------+---------------------------------------------+-----------------------------------------------+-------------+-------+----------+------+------------+-------------+--------------+------------+----------------+-----------+-----------+---------------+----------------+--------------+------------+
|CustomerID|Churn|InternetService_index_ohe_Fiber_optic|InternetService_index_ohe_DSL|InternetService_index_ohe_No|Contract_index_ohe_Month_to_month|Contract_index_ohe_Two_year|Contract_index_ohe_One_year|PaymentMethod_index_ohe_Electronic_check|PaymentMethod_index_ohe_Mailed_check|PaymentMethod_index_ohe_Credit_card_automatic|PaymentMethod_index_ohe_Bank_transfer_automatic|SeniorCitizen|Partner|Dependents|tenure|PhoneSer

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier

# StringIndexer
indexer = StringIndexer(inputCol="Churn", outputCol="label")

# Assembler
feature_cols = [col for col in churn.columns if col not in ['CustomerID', 'Churn']]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid="skip")

# RandomForest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100, maxDepth=10, seed=42)

# Full pipeline
full_pipeline = Pipeline(stages=[indexer, assembler, rf])

# Split data
train, test = churn.randomSplit([0.8, 0.2], seed=42)

# Fit pipeline
pipeline_model = full_pipeline.fit(train)

25/08/21 12:12:00 WARN DAGScheduler: Broadcasting large task binary with size 1624.6 KiB
25/08/21 12:12:01 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
25/08/21 12:12:02 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
25/08/21 12:12:03 WARN DAGScheduler: Broadcasting large task binary with size 6.1 MiB


In [13]:
pipeline_model.transform(test).select("features", "label", "prediction").show(10)

+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|(25,[0,3,6,13,14,...|  1.0|       1.0|
|[1.0,0.0,0.0,0.0,...|  0.0|       0.0|
|(25,[1,3,6,10,13,...|  0.0|       0.0|
|(25,[0,4,9,13,14,...|  0.0|       0.0|
|(25,[1,3,6,10,13,...|  1.0|       1.0|
|(25,[2,3,7,13,14,...|  0.0|       0.0|
|(25,[2,4,9,11,12,...|  0.0|       0.0|
|(25,[2,4,6,12,13,...|  0.0|       0.0|
|(25,[1,4,9,11,13,...|  0.0|       0.0|
|(25,[1,5,8,13,14,...|  0.0|       0.0|
+--------------------+-----+----------+
only showing top 10 rows


25/08/21 12:12:15 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


In [14]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Get predictions
predictions = pipeline_model.transform(test)

# AUC (area under ROC)
binary_eval = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = binary_eval.evaluate(predictions)
print(f"AUC: {auc}")
# Accuracy
accuracy_eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = accuracy_eval.evaluate(predictions)
print(f"Accuracy: {accuracy}")

# F1 Score
f1_eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = f1_eval.evaluate(predictions)
print(f"F1 Score: {f1}")

25/08/21 12:12:23 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


AUC: 0.8444625112917842


25/08/21 12:12:24 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


Accuracy: 0.8013392857142857


25/08/21 12:12:24 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB


F1 Score: 0.791278096174227


In [15]:
# Save the pipeline model
pipeline_model.write().overwrite().save(f"{ROOT_DIR}/models/churn_pipeline")

In [16]:
spark.stop()